In [10]:
import requests
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os


def extract_data():
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    return df

df = extract_data()
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,91849.000000,1834889150795,1,1834890070532,9.007215e+10,95904.000000,...,1.995016e+07,2.100000e+07,126080.000000,-27.13251,2025-10-06T18:57:42.558Z,67.810000,1.353853e+05,2013-07-06T00:00:00.000Z,None,2025-11-17T19:43:23.107Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2993.810000,362591979896,2,362591979896,3.766678e+10,3215.640000,...,1.206962e+08,NaN,4946.050000,-39.36713,2025-08-24T19:21:03.333Z,0.432979,6.925276e+05,2015-10-20T00:00:00.000Z,"{'times': 42.580714581066225, 'currency': 'btc...",2025-11-17T19:43:23.739Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,0.999021,183934528712,3,189398318201,1.295151e+11,0.999833,...,1.895074e+11,NaN,1.320000,-24.48883,2018-07-24T00:00:00.000Z,0.572521,7.450622e+01,2015-03-02T00:00:00.000Z,None,2025-11-17T19:43:25.776Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.140000,128637575673,4,213727804335,5.548408e+09,2.280000,...,9.998576e+10,1.000000e+11,3.650000,-41.45411,2025-07-18T03:40:53.808Z,0.002686,7.937283e+04,2014-05-22T00:00:00.000Z,None,2025-11-17T19:43:25.590Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,899.370000,123973195884,5,123973195011,3.045218e+09,937.110000,...,1.377369e+08,2.000000e+08,1369.990000,-34.35200,2025-10-13T08:41:24.131Z,0.039818,2.258624e+06,2017-10-19T00:00:00.000Z,None,2025-11-17T19:43:22.871Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,kaspa,kas,Kaspa,https://coin-images.coingecko.com/coins/images...,0.041824,1120144486,96,1126415921,4.341047e+07,0.045873,...,2.692707e+10,2.870403e+10,0.207411,-79.80389,2024-08-01T00:40:47.164Z,0.000171,2.438937e+04,2022-05-26T14:42:59.316Z,None,2025-11-17T19:43:14.591Z
96,ignition-fbtc,fbtc,Function FBTC,https://coin-images.coingecko.com/coins/images...,91442.000000,1093987865,97,1093987865,4.567390e+05,97489.000000,...,1.194764e+04,NaN,127954.000000,-28.09647,2025-10-06T12:50:40.428Z,11469.830000,7.021351e+02,2024-09-10T18:18:02.102Z,None,2025-11-17T19:43:25.939Z
97,ripple-usd,rlusd,Ripple USD,https://coin-images.coingecko.com/coins/images...,0.999533,1088869439,98,1088869439,8.206527e+07,1.001000,...,1.089198e+09,NaN,1.073000,-6.80582,2024-12-26T10:45:52.337Z,0.962292,3.898200e+00,2024-12-18T04:40:40.880Z,None,2025-11-17T19:43:14.629Z
98,hashnote-usyc,usyc,Circle USYC,https://coin-images.coingecko.com/coins/images...,1.110000,1076504664,99,1076504664,5.519000e+04,1.110000,...,9.733448e+08,NaN,1.120000,-0.97026,2025-03-04T19:26:50.778Z,1.026000,7.785000e+00,2024-12-05T22:31:20.475Z,None,2025-11-17T12:22:28.323Z


In [12]:
def transform_data(df):
    
    df = df[['id', 'symbol', 'name', 'current_price', 'market_cap', 'total_volume', 'last_updated']]
    
    df = df.dropna()
    
    df['loaded_at'] = pd.Timestamp.now()
    
    return df

transform_data(df)

,id,symbol,name,current_price,market_cap,total_volume,last_updated,loaded_at
0,bitcoin,btc,Bitcoin,91849.000000,1834889150795,9.007215e+10,2025-11-17T19:43:23.107Z,2025-11-18 02:45:05.737157
1,ethereum,eth,Ethereum,2993.810000,362591979896,3.766678e+10,2025-11-17T19:43:23.739Z,2025-11-18 02:45:05.737157
2,tether,usdt,Tether,0.999021,183934528712,1.295151e+11,2025-11-17T19:43:25.776Z,2025-11-18 02:45:05.737157
3,ripple,xrp,XRP,2.140000,128637575673,5.548408e+09,2025-11-17T19:43:25.590Z,2025-11-18 02:45:05.737157
4,binancecoin,bnb,BNB,899.370000,123973195884,3.045218e+09,2025-11-17T19:43:22.871Z,2025-11-18 02:45:05.737157
...,...,...,...,...,...,...,...,...
95,kaspa,kas,Kaspa,0.041824,1120144486,4.341047e+07,2025-11-17T19:43:14.591Z,2025-11-18 02:45:05.737157
96,ignition-fbtc,fbtc,Function FBTC,91442.000000,1093987865,4.567390e+05,2025-11-17T19:43:25.939Z,2025-11-18 02:45:05.737157
97,ripple-usd,rlusd,Ripple USD,0.999533,1088869439,8.206527e+07,2025-11-17T19:43:14.629Z,2025-11-18 02:45:05.737157
98,hashnote-usyc,usyc,Circle USYC,1.110000,1076504664,5.519000e+04,2025-11-17T12:22:28.323Z,2025-11-18 02:45:05.737157
